In [ ]:
import pandas as pd
import numpy as np
import pyspark as py
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder \
 .master('local[*]') \
 .appName('Spark NLP') \
 .config('spark.driver.memory', '6g') \
 .config('spark.executor.memory', '6g') \
 .config('spark.jars.packages', 'JohnSnowLabs:spark-nlp:2.5.2') \
 .config('spark.kryoserializer.buffer.max', '600M') \
 .getOrCreate()
   

sc = spark.sparkContext
sqlContext = SQLContext(sc)

df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', delimiter="\t").load('test-dataset.csv')

In [ ]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
import preprocess

comments = df.select(
    'Comments'
)
preprocessed = preprocess.Preprocess(comments)
prep_df = preprocessed.get_dataframe()

In [ ]:
import classify

lda_category_n8w7p250_lookup = {
    0: [1],
    1: [3, 8],
    2: [9, 12],
    3: [11, 12],
    4: [4, 7],
    5: [13],
    6: [8, 5, 6],
    7: [2, 4, 7]
}

classified = classify.UnsupervisedClassification(prep_df, lda_category_n8w7p250_lookup, topics=8, passes=250, decay=None, iterations=None)